In [2]:
pip install folium

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install geopy

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install requests

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder

# Load the dataset
df = pd.read_csv('large_routes_features.csv')

# Data Preprocessing
# Convert categorical features into numeric labels (e.g., street lighting, CCTV presence)
label_encoder = LabelEncoder()
df['StreetLighting'] = label_encoder.fit_transform(df['StreetLighting'])
df['PedestrianTraffic'] = label_encoder.fit_transform(df['PedestrianTraffic'])
df['CCTVPresence'] = label_encoder.fit_transform(df['CCTVPresence'])
df['SidewalkCondition'] = label_encoder.fit_transform(df['SidewalkCondition'])

# Define features (X) and target variable (y)
X = df[['Latitude', 'Longitude', 'CrimeFrequency', 'CrimeSeverity', 'StreetLighting', 
        'PedestrianTraffic', 'CCTVPresence', 'PoliceProximity', 'PublicTransportProximity', 'SidewalkCondition']]

# Creating a new target variable 'SafetyLevel' (Categorizing Safety Level based on custom rules)
# You can customize these thresholds based on real data
df['SafetyLevel'] = pd.cut(df['CrimeSeverity'], bins=[0, 3, 6, 10], labels=['Safe', 'Medium', 'Dangerous'])

y = df['SafetyLevel']

# Split data into train and test sets (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [3]:
# Train a Random Forest Classifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Predict on the test set
y_pred = rf_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

# Print classification report for detailed metrics
print(classification_report(y_test, y_pred))


Accuracy: 100.00%
              precision    recall  f1-score   support

   Dangerous       1.00      1.00      1.00        76
      Medium       1.00      1.00      1.00        69
        Safe       1.00      1.00      1.00        55

    accuracy                           1.00       200
   macro avg       1.00      1.00      1.00       200
weighted avg       1.00      1.00      1.00       200



In [5]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

# Encode target labels (SafetyLevel)
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Optionally, you can one-hot encode the labels (not needed for sparse_categorical_crossentropy)
# y_train_encoded = to_categorical(y_train_encoded, num_classes=3)
# y_test_encoded = to_categorical(y_test_encoded, num_classes=3)

# Build a Neural Network Model (Feedforward)
model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))  # Input layer
model.add(Dense(32, activation='relu'))  # Hidden layer
model.add(Dense(3, activation='softmax'))  # Output layer (3 classes: Safe, Medium, Dangerous)

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train_encoded, epochs=20, batch_size=32, validation_split=0.2)

# Evaluate on test set
loss, accuracy = model.evaluate(X_test, y_test_encoded)
print(f"Test Accuracy: {accuracy * 100:.2f}%")


Epoch 1/20
20/20 [==============================] - 1s 22ms/step - loss: 98.0717 - accuracy: 0.3234 - val_loss: 39.6873 - val_accuracy: 0.3625
Epoch 2/20
20/20 [==============================] - 0s 2ms/step - loss: 27.8569 - accuracy: 0.3156 - val_loss: 14.4439 - val_accuracy: 0.3250
Epoch 3/20
20/20 [==============================] - 0s 7ms/step - loss: 8.3234 - accuracy: 0.3703 - val_loss: 5.1728 - val_accuracy: 0.3250
Epoch 4/20
20/20 [==============================] - 0s 12ms/step - loss: 3.5116 - accuracy: 0.4094 - val_loss: 3.6215 - val_accuracy: 0.3750
Epoch 5/20
20/20 [==============================] - 0s 6ms/step - loss: 2.8445 - accuracy: 0.4391 - val_loss: 2.4156 - val_accuracy: 0.4375
Epoch 6/20
20/20 [==============================] - 0s 10ms/step - loss: 2.0421 - accuracy: 0.4391 - val_loss: 2.5635 - val_accuracy: 0.4125
Epoch 7/20
20/20 [==============================] - 0s 10ms/step - loss: 1.6969 - accuracy: 0.4734 - val_loss: 1.7802 - val_accuracy: 0.4437
Epoch 8/20
2

In [7]:
from sklearn.preprocessing import LabelEncoder

# Ensure the LabelEncoder is fitted with all possible values during training
street_lighting_classes = ['Good', 'Fair', 'Poor']
pedestrian_traffic_classes = ['Low', 'Medium', 'High']
cctv_presence_classes = ['Yes', 'No']
sidewalk_condition_classes = ['Good', 'Fair', 'Poor']

# Fit the label encoder with all possible values
street_lighting_encoder = LabelEncoder()
street_lighting_encoder.fit(street_lighting_classes)

pedestrian_traffic_encoder = LabelEncoder()
pedestrian_traffic_encoder.fit(pedestrian_traffic_classes)

cctv_presence_encoder = LabelEncoder()
cctv_presence_encoder.fit(cctv_presence_classes)

sidewalk_condition_encoder = LabelEncoder()
sidewalk_condition_encoder.fit(sidewalk_condition_classes)

# Predict safety levels for new routes (make sure to use the same encoders)
new_routes = pd.DataFrame({
    'Latitude': [13.1067, 13.1100],
    'Longitude': [80.0970, 80.1200],
    'CrimeFrequency': [5, 7],
    'CrimeSeverity': [2, 6],
    'StreetLighting': street_lighting_encoder.transform(['Good', 'Fair']),
    'PedestrianTraffic': pedestrian_traffic_encoder.transform(['Medium', 'High']),
    'CCTVPresence': cctv_presence_encoder.transform(['Yes', 'No']),
    'PoliceProximity': [300, 1000],
    'PublicTransportProximity': [500, 1200],
    'SidewalkCondition': sidewalk_condition_encoder.transform(['Good', 'Poor'])
})

# Predict safety level for these new routes
new_routes_predictions = rf_model.predict(new_routes)
print("Predicted Safety Levels for New Routes: ", new_routes_predictions)


Predicted Safety Levels for New Routes:  ['Safe' 'Medium']


In [8]:
# Add a marker to visualize the predicted safety levels
for i, row in new_routes.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=f"Predicted Safety Level: {new_routes_predictions[i]}",
        icon=folium.Icon(color='green' if new_routes_predictions[i] == 'Safe' else 'red')
    ).add_to(mymap)

# Show the map
mymap
